In [1]:
from google.colab import drive

drive.mount('/content/gdrive/')

import sys
sys.path.append('/content/gdrive/My Drive/Sleep_Data_Imputation_Handling/data')
import os
os.chdir('/content/gdrive/My Drive/Sleep_Data_Imputation_Handling/data')

Mounted at /content/gdrive/


In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('synthetic_data.csv').drop(columns=['Unnamed: 0'])

In [3]:
data 

,timespan,SleepStatus,actigraphy,light,Schedule,detailed_sleepwake
0,0,Wake,460.0,0.33,0,1.0
1,1,Wake,262.0,0.66,0,1.0
2,2,Wake,207.0,0.66,0,1.0
3,3,Wake,490.0,0.75,0,1.0
4,4,Wake,230.0,1.96,0,1.0
...,...,...,...,...,...,...
114229,1435,Wake,603.0,29.64,1,1.0
114230,1436,Wake,398.0,0.00,1,1.0
114231,1437,Wake,385.0,12.60,1,1.0
114232,1438,Wake,182.0,21.71,1,1.0


In [4]:
import random

In [12]:
Data = data.copy()
Data['denote'] = Data['actigraphy'].apply(lambda x: 0. if np.isnan(x) else 1.)
for i in range(50):
  getrandom = random.randint(0, Data.shape[0])
  xd = np.random.gamma(8, 13, 1)[0]
  Data.loc[getrandom: min(getrandom+xd, Data.shape[0]), 'denote'] = 0

Data.loc[Data['denote'] == 0, 'actigraphy'] = np.nan

In [13]:
Data['Label'] = Data['SleepStatus'].apply(lambda x: 0 if x == 'Sleep' else 1.)

fillmisscol = np.full((Data.shape[0]), 0)
i = 0
while i < Data.shape[0]:
  if Data['denote'][i] == 0:
    counter = 1
    while i + counter < Data.shape[0] and Data['denote'][i + counter] == 0:
      counter += 1
    fillmisscol[i:i+counter] = counter
    i = i + counter
  else:
    i += 1

Data['type'] = fillmisscol // 30 + 1
Data.loc[ fillmisscol==0 , 'type'] = 0

In [14]:
for i in [[1,2], [3, 4, 5, 6], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]]:
      print('------------------------------------')
      all0 = Data[(Data['Label'] == 0) & (Data['type'].isin(i)) ].shape[0]
      all1 = Data[(Data['Label'] == 1) & (Data['type'].isin(i))].shape[0]

      print('Type ', [[1,2], [3, 4, 5, 6], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]].index(i) + 1)
      print('Total_sleep: ', all0)
      
      print('Total_wake: ',  all1)


------------------------------------
Type  1
Total_sleep:  152
Total_wake:  1608
------------------------------------
Type  2
Total_sleep:  1049
Total_wake:  3963
------------------------------------
Type  3
Total_sleep:  99
Total_wake:  1468


In [15]:
Data.drop(columns=['denote', 'Label', 'type']).to_csv('synthetic_data2.csv')